In [ ]:
import geopandas as gpd
import plotly.graph_objects as go
from shapely.geometry import Polygon, MultiPolygon

# load
gdf = gpd.read_file("/home/rin/code/30daymapchallenge2025/data/Ecoregions2017/Ecoregions2017.json")
gdf = gdf[gdf.geometry.notnull()].reset_index(drop=True)

biome_colors = {
    "Boreal Forests/Taiga":                          "rgb(98,135,173)",
    "Deserts & Xeric Shrublands":                    "rgb(214,192,94)",
    "Flooded Grasslands & Savannas":                 "rgb(151,216,210)",
    "Mangroves":                                     "rgb(255,0,255)",
    "Mediterranean Forests, Woodlands & Scrub":      "rgb(209,94,75)",
    "Montane Grasslands & Shrublands":               "rgb(205,189,159)",
    "Rock and Ice":                                  "rgb(240,240,240)",
    "Temperate Broadleaf & Mixed Forests":           "rgb(81,135,109)",
    "Temperate Conifer Forests":                     "rgb(35,103,70)",
    "Temperate Grasslands, Savannas & Shrublands":   "rgb(201,229,145)",
    "Tropical & Subtropical Coniferous Forests":     "rgb(21,80,57)",
    "Tropical & Subtropical Dry Broadleaf Forests":  "rgb(216,175,74)",
    "Tropical & Subtropical Grasslands, Savannas & Shrublands": "rgb(127,212,130)",
    "Tropical & Subtropical Moist Broadleaf Forests": "rgb(68,159,101)",
    "Tundra":                                        "rgb(192,233,221)",
}

# fallback
get_color = lambda b: biome_colors.get(b, "rgb(200,200,200)")

# flatten all polys into lists
lons = []
lats = []
colors = []
texts = []

for _, row in gdf.iterrows():
    geom = row.geometry
    biome = row.get("BIOME_NAME", "")
    color = get_color(biome)
    label = biome

    if geom.geom_type == "Polygon":
        polys = [geom]
    elif geom.geom_type == "MultiPolygon":
        polys = geom.geoms
    else:
        continue

    for poly in polys:
        if poly is None or poly.exterior is None:
            continue
        x, y = poly.exterior.coords.xy
        xs = list(x)
        ys = list(y)

        # append polygon
        lons += xs + [None]
        lats += ys + [None]
        colors.append(color)
        texts.append(label)



fig = go.Figure(
    go.Scattergeo(
        lon=lons,
        lat=lats,
        mode="lines",
        line=dict(color="black", width=0.3),
        fill="toself",
        fillcolor="rgba(0,0,0,0)",   # must set, but we fake fill via overlay
        hoverinfo="none",
    )
)



fig.update_geos(
    projection_type="orthographic",
    showcountries=False,
    showcoastlines=False,
    showland=True,
    landcolor="rgb(235,240,245)",
    showocean=True,
    oceancolor="rgb(200,210,230)",
)

fig.update_layout(
    showlegend=False,
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.show()
